In [6]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import torch
import pandas as pd


In [8]:
data = pd.read_excel('Отзывы_спам,_модерация,_проблемы,_корректные.xlsx')
data.rename(columns={'Unnamed: 0': 'Тип отзыва','Unnamed: 1' : 'Текст отзыва', 'Unnamed: 2': 'Спам'}, inplace=True)
data

,Тип отзыва,Текст отзыва,Спам
0,позитивный,👍👍👍Супер разъ*б 👍👍👍👍👍,"Спам, удалить"
1,позитивный,Лазила лаповое каша а прилип пиал,"Спам, удалить"
2,позитивный,Ну что как у нас с вами получилось с этим рабо...,"Спам, удалить"
3,позитивный,Ну что там как там дела как настроение у,"Спам, удалить"
4,позитивный,Ну что там как там дела как настроение у вас ч...,"Спам, удалить"
...,...,...,...
2209,негативный,Ужасно.\nЖдали несколько часов \nНичего так и ...,Не доставлен
2210,негативный,Ужасный сервис !!!\nДоставка ехала почти три ч...,Не довезли
2211,позитивный,Акция пицца в подарок не работает,Другое
2212,позитивный,не увидел имбиря😔\nне увидел васаби😩,Уточнения по отзывам (допы)


In [9]:
data['Спам'] = data['Спам'].apply(lambda x: 1 if x == 'Спам, удалить' else 0)
data['Тип отзыва'] = data['Тип отзыва'].apply(lambda x: 1 if x == 'позитивный' else 0)
data

,Тип отзыва,Текст отзыва,Спам
0,1,👍👍👍Супер разъ*б 👍👍👍👍👍,1
1,1,Лазила лаповое каша а прилип пиал,1
2,1,Ну что как у нас с вами получилось с этим рабо...,1
3,1,Ну что там как там дела как настроение у,1
4,1,Ну что там как там дела как настроение у вас ч...,1
...,...,...,...
2209,0,Ужасно.\nЖдали несколько часов \nНичего так и ...,0
2210,0,Ужасный сервис !!!\nДоставка ехала почти три ч...,0
2211,1,Акция пицца в подарок не работает,0
2212,1,не увидел имбиря😔\nне увидел васаби😩,0


In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import Dataset as HFDataset, DatasetDict  # Hugging Face Datasets

# Загрузка модели RuBERT
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=len(data['Спам'].unique()))

# Лемматизатор
morph = MorphAnalyzer()

def preprocess_text(text):
    # Убираем спецсимволы, переводим в нижний регистр
    text = re.sub(r"[^\w\s]", "", text.lower())
    # Исправляем пробелы и сокращения
    text = re.sub(r"\s+", " ", text).strip()
    # Лемматизация
    words = text.split()
    lemmatized_text = " ".join(morph.parse(word)[0].normal_form for word in words)
    return lemmatized_text

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {"text": self.texts[idx], "label": self.labels[idx]}

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Разделение данных
X = data['Текст отзыва'].apply(preprocess_text).tolist()
y = data['Спам'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Подготовка данных в формате Hugging Face
train_dataset = HFDataset.from_dict({"text": X_train, "label": y_train})
test_dataset = HFDataset.from_dict({"text": X_test, "label": y_test})

# Токенизация
tokenized_datasets = DatasetDict({
    "train": train_dataset.map(tokenize_function, batched=True),
    "test": test_dataset.map(tokenize_function, batched=True),
})

# Удаление сырых текстов (они больше не нужны после токенизации)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Определение Trainer
training_args = TrainingArguments(
    output_dir="./results",          # Куда сохранять результаты
    evaluation_strategy="epoch",    # Оценивать после каждой эпохи
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',           # Логи обучения
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Обучение модели
trainer.train()

# Оценка модели
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1771 [00:00<?, ? examples/s]

Map:   0%|          | 0/443 [00:00<?, ? examples/s]

/home/student-math-01/anaconda3/envs/py33/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_423266/2958757075.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",          # Куда сохранять результаты
#     evaluation_strategy="epoch",    # Оценивать после каждой эпохи
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_strategy="epoch",
#     logging_dir='./logs',           # Логи обучения
#     logging_steps=10,
# )


/home/student-math-01/anaconda3/envs/py33/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     tokenizer=tokenizer,
# )


/tmp/ipykernel_383190/817050193.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
# trainer.train()

Epoch,Training Loss,Validation Loss
1,0.183300,0.145144
2,0.004400,0.126015
3,0.094700,0.169506


TrainOutput(global_step=333, training_loss=0.07242698527246087, metrics={'train_runtime': 1165.0184, 'train_samples_per_second': 4.56, 'train_steps_per_second': 0.286, 'total_flos': 349477259281920.0, 'train_loss': 0.07242698527246087, 'epoch': 3.0})

In [ ]:
# metrics = trainer.evaluate()
# print(metrics)

{'eval_loss': 0.16950581967830658, 'eval_runtime': 24.6686, 'eval_samples_per_second': 17.958, 'eval_steps_per_second': 1.135, 'epoch': 3.0}


In [ ]:
# from sklearn.metrics import classification_report, accuracy_score

# # Получение предсказаний на валидационном датасете
# predictions = trainer.predict(tokenized_datasets["validation"])
# pred_labels = predictions.predictions.argmax(axis=1)
# true_labels = predictions.label_ids

# # Вывод точности
# accuracy = accuracy_score(true_labels, pred_labels)
# print("Accuracy:", accuracy)

# # Подробный отчет о классификации
# report = classification_report(true_labels, pred_labels, target_names=["Class 0", "Class 1"])
# print(report)


Accuracy: 0.9661399548532731
              precision    recall  f1-score   support

     Class 0       0.98      0.98      0.98       389
     Class 1       0.87      0.85      0.86        54

    accuracy                           0.97       443
   macro avg       0.92      0.92      0.92       443
weighted avg       0.97      0.97      0.97       443



In [ ]:
import torch

# Упаковываем модель и токенизатор в словарь
save_path = "model1.pth"
torch.save({
    "model_state_dict": model.state_dict(),
    "tokenizer": tokenizer,
    "config": model.config,
}, save_path)
print(f"Модель сохранена в {save_path}")


Модель сохранена в model.pth


In [ ]:
y = data['Тип отзыва'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Подготовка данных в формате Hugging Face
train_dataset = HFDataset.from_dict({"text": X_train, "label": y_train})
test_dataset = HFDataset.from_dict({"text": X_test, "label": y_test})

# Токенизация
tokenized_datasets = DatasetDict({
    "train": train_dataset.map(tokenize_function, batched=True),
    "test": test_dataset.map(tokenize_function, batched=True),
})

# Удаление сырых текстов (они больше не нужны после токенизации)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Определение Trainer
training_args = TrainingArguments(
    output_dir="./results",          # Куда сохранять результаты
    evaluation_strategy="epoch",    # Оценивать после каждой эпохи
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',           # Логи обучения
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Обучение модели
trainer.train()

# Оценка модели
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

In [ ]:
import torch

# Упаковываем модель и токенизатор в словарь
save_path = "model2.pth"
torch.save({
    "model_state_dict": model.state_dict(),
    "tokenizer": tokenizer,
    "config": model.config,
}, save_path)
print(f"Модель сохранена в {save_path}")
